In [3]:
from torchvision import models
from torchinfo import summary   # 모델 구조 확인하기 위해 torchinfo 라이브러리

# 알렉스넷 모델 불러오기 함수 (weights=사전 학습된 가중치 매개변수)
model = models.alexnet(weights="AlexNet_Weights.IMAGENET1K_V1")
# 모델 요약 함수 - torchinfo.summary(model, input_data)
summary(model, (1, 3, 224, 224), device="cpu")

Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /Users/seoyun/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:03<00:00, 71.2MB/s] 


Layer (type:depth-idx)                   Output Shape              Param #
AlexNet                                  [1, 1000]                 --
├─Sequential: 1-1                        [1, 256, 6, 6]            --
│    └─Conv2d: 2-1                       [1, 64, 55, 55]           23,296
│    └─ReLU: 2-2                         [1, 64, 55, 55]           --
│    └─MaxPool2d: 2-3                    [1, 64, 27, 27]           --
│    └─Conv2d: 2-4                       [1, 192, 27, 27]          307,392
│    └─ReLU: 2-5                         [1, 192, 27, 27]          --
│    └─MaxPool2d: 2-6                    [1, 192, 13, 13]          --
│    └─Conv2d: 2-7                       [1, 384, 13, 13]          663,936
│    └─ReLU: 2-8                         [1, 384, 13, 13]          --
│    └─Conv2d: 2-9                       [1, 256, 13, 13]          884,992
│    └─ReLU: 2-10                        [1, 256, 13, 13]          --
│    └─Conv2d: 2-11                      [1, 256, 13, 13]         

In [4]:
####### 클래스 정보 불러오기
with open("../datasets/imagenet_classes.txt", "r") as file:
    classes = file.read().splitlines()
    
print(f"클래스 개수 : {len(classes)}")
print(f"첫 번째 클래스 레이블 : {classes[0]}")

클래스 개수 : 1000
첫 번째 클래스 레이블 : tench


In [8]:
####### 이미지 데이터 전처리 (알렉스넷에서 학습한 이미지 데이터와 동일한 형태로 전처리 수행)
import torch
from PIL import Image
from torchvision import models, transforms

# 전처리 (입력이미지 = 256x256)
# RGB 픽셀값의 평균과 분산을 활용해 정규화를 적용한다,,?
transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),  # 0~1로 픽셀값 스케일링
        transforms.Normalize(   # ImageNet 데이터셋에 대해 계산된 값
            mean = [0.485, 0.456, 0.406],
            std  = [0.229, 0.224, 0.225]
        ),
    ]
)

device = "mps" if torch.backends.mps.is_available() and torch.backends.mps.is_built() else "cpu"
model  = models.alexnet(weights="AlexNet_Weights.IMAGENET1K_V1").eval().to(device)

tensors = []
files   = ["../datasets/images/airplane.jpg", "../datasets/images/bus.jpg"]
for file in files: 
    image = Image.open(file)
    tensors.append(transform(image))

tensors = torch.stack(tensors)  # 모든 이미지 텐서를 하나의 배치로 결합
print(f"입력 텐서의 크기 : {tensors.shape}")

입력 텐서의 크기 : torch.Size([2, 3, 224, 224])


In [11]:
####### 알렉스넷 모델 추론
# 모델과 데이터를 활용해 추론해 보고 상위 다섯 개의 클래스와 예측 확률을 출력
import numpy as np
from torch.nn import functional as F

# 기울기 계산 없이 순전파 계산
with torch.no_grad(): 
    outputs = model(tensors.to(device))
    probs   = F.softmax(outputs, dim=-1)
    print(probs.size())   # [배치크기(이미지개수), 클래스개수]
    # 텐서값이 가장 큰 상위 다섯 개의 요소 반환
    top_probs, top_idxs = probs.topk(5)

top_probs   = top_probs.detach().cpu().numpy()  # 상위 다섯 개 확률
top_idxs    = top_idxs.detach().cpu().numpy()   # 상위 다섯 개 인덱스 (숫자)
top_classes = np.array(classes)[top_idxs]       # 상위 다섯 개 인덱스 이름 (class명)

for idx, (cls, prob) in enumerate(zip(top_classes, top_probs)): 
    print(f"{files[idx]} 추론 결과")
    for c, p in zip(cls, prob): 
        print(f" - {c:<30} : {p * 100:>5.2f}%")

torch.Size([2, 1000])
../datasets/images/airplane.jpg 추론 결과
 - airliner                       : 66.83%
 - warplane                       : 20.12%
 - wing                           :  9.29%
 - space shuttle                  :  2.89%
 - missile                        :  0.38%
../datasets/images/bus.jpg 추론 결과
 - streetcar                      : 60.59%
 - trolleybus                     : 37.66%
 - minibus                        :  1.54%
 - passenger car                  :  0.17%
 - recreational vehicle           :  0.03%


dim=-1의 의미:  
dim=-1은 텐서의 마지막 차원을 의미합니다. 이는 주로 신경망에서 출력 벡터가 마지막 차원에 위치하는 구조에서 사용됩니다.  
예를 들어, outputs 텐서의 크기가 [배치 크기, 클래스 수]라면, dim=-1은 각 예측 벡터(클래스 수에 해당하는 차원)에 대해 softmax를 적용함을 의미합니다. 즉, 각 배치 내의 데이터 포인트에 대해 독립적으로 클래스 별 확률을 계산하게 됩니다.